In [10]:
from components import *
import numpy as np
import scipy.signal as sci
import time

In [70]:
def benchmark(layer, x_shape, delta_shape, samples = 50, resolution = 200, digits_precision = 4):
    def measure_func(func, input_shape):
        times = []

        for _ in range(samples):
            now = time.time()
            for i in range(resolution):
                input = np.random.randn(*input_shape)
                func(input)
            times.append(time.time() - now)

        mean = 1000 * np.mean(times) / resolution
        median = 1000 * np.median(times) / resolution
        std = 1000 * np.std(times) / np.sqrt(resolution)

        return mean, median, std

    mean_ffwd, median_ffwd, std_ffwd = measure_func(layer.feedforward, x_shape)
    mean_back, median_back, std_back = measure_func(layer.backprop, delta_shape)

    print(f"Feedforward: {round(mean_ffwd, digits_precision)} ± {round(std_ffwd, digits_precision)} ms         median {median_ffwd} ms")
    print(f"Backprop: {round(mean_back, digits_precision)} ± {round(std_back, digits_precision)} ms         median {median_back} ms")

In [71]:
get_optimizer = lambda : SGD_momentum_optimizer(0.001, 20, 0.99).get_optimizer()

In [ ]:
# FullyConnected benchmark
f = FullyConnected(100, 100, tanh(), L2Regularization(0.001))
f.set_optimizer(get_optimizer())
benchmark(f, (100, 1), (100, 1))

Feedforward: 0.0839 ± 0.3621 ms         median 0.08337914943695068 ms
Backprop: 0.2043 ± 0.6295 ms         median 0.18944263458251953 ms


In [75]:
f = FullyConnected(200, 200, tanh(), L2Regularization(0.001))
f.set_optimizer(get_optimizer())
benchmark(f, (200, 1), (200, 1))

Feedforward: 0.0985 ± 0.6751 ms         median 0.08848786354064941 ms
Backprop: 0.5042 ± 2.2526 ms         median 0.44431984424591064 ms


In [77]:
conv = Convolution((4, 28, 28), (3, 3), 8, tanh())
conv.set_optimizer(get_optimizer())
benchmark(conv, (10, 4, 28, 28), (10, 8, 26, 26), resolution=20)

Feedforward: 35.6962 ± 38.3102 ms         median 33.080703020095825 ms
Backprop: 77.7228 ± 23.1754 ms         median 78.09931635856628 ms


In [89]:
maxpool = MaxPool((4, 28, 28))
benchmark(maxpool, (10, 4, 28, 28), (10, 4, 14, 14), resolution=20)

Feedforward: 71.5669 ± 79.2425 ms         median 64.68868255615234 ms
Backprop: 9.5479 ± 2.0161 ms         median 9.358930587768555 ms
